# Infer-14-Decision-Utility-Foundations : Axiomes et Fondements

**Serie** : Programmation Probabiliste avec Infer.NET (14/20)  
**Duree estimee** : 50 minutes  
**Prerequis** : Notebooks 1-8 (modeles probabilistes de base)

---

## Objectifs

- Comprendre les **loteries** comme representation des choix en environnement stochastique
- Maitriser les **axiomes de Von Neumann-Morgenstern**
- Deriver la **fonction d'utilite** par calibration
- Comprendre la **maximisation de l'utilite esperee** comme fondement de l'**agent rationnel**

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-13-Debugging](Infer-13-Debugging.ipynb) | [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb) |

---

## 1. Introduction : Pourquoi l'Utilite ?

### Le probleme de la decision sous incertitude

Jusqu'ici, nous avons utilise Infer.NET pour calculer des distributions de probabilite. Mais comment **agir** face a l'incertitude ?

Considerez ce dilemme :

| Option | Description |
|--------|-------------|
| A | Gagner 100 EUR avec certitude |
| B | 50% de chance de gagner 200 EUR, 50% de ne rien gagner |

**Valeur esperee** : E[A] = 100, E[B] = 100. Identiques !

Pourtant, la plupart des gens preferent A. Pourquoi ?

### La valeur esperee ne suffit pas

Le **Paradoxe de Saint-Petersbourg** (1713) illustre cette limite :

> Une piece est lancee jusqu'a obtenir Face. Si Face apparait au tour n, vous gagnez 2^n euros.
> Combien paieriez-vous pour jouer ?

$$E[\text{gain}] = \sum_{n=1}^{\infty} \frac{1}{2^n} \times 2^n = \sum_{n=1}^{\infty} 1 = \infty$$

Valeur esperee infinie, mais personne ne paierait un million pour jouer. Il faut une autre approche.

## 2. Loteries : Representation Formelle des Choix

### Definition

Une **loterie** est une distribution de probabilite sur des **outcomes** (resultats).

**Notation** : L = [p₁:o₁, p₂:o₂, ..., pₙ:oₙ]

Ou :
- pᵢ est la probabilite de l'outcome oᵢ
- Σpᵢ = 1

### Exemples

| Loterie | Description |
|---------|-------------|
| [1.0 : 100EUR] | 100 EUR avec certitude (outcome degenere) |
| [0.5 : 200EUR, 0.5 : 0EUR] | Pile ou face pour 200 EUR |
| [0.6 : Succes, 0.4 : Echec] | Examen avec 60% de reussite |

### Loteries composees

Une **loterie composee** a d'autres loteries comme outcomes :

L = [0.5 : L₁, 0.5 : L₂]

Elle peut etre **reduite** en loterie simple par le calcul des probabilites totales.

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge pour Decision Theory !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge pour Decision Theory !


In [2]:
// Representation d'une loterie simple
public class Loterie<T>
{
    public List<(double prob, T outcome)> Outcomes { get; }
    
    public Loterie(params (double prob, T outcome)[] items)
    {
        Outcomes = items.ToList();
        var total = Outcomes.Sum(x => x.prob);
        if (Math.Abs(total - 1.0) > 1e-6)
            throw new ArgumentException($"Probabilites doivent sommer a 1, got {total}");
    }
    
    public override string ToString()
    {
        var parts = Outcomes.Select(o => $"{o.prob:P0}:{o.outcome}");
        return $"[{string.Join(", ", parts)}]";
    }
}

// Exemples
var certainty = new Loterie<double>((1.0, 100));
var coinFlip = new Loterie<double>((0.5, 200), (0.5, 0));

Console.WriteLine($"Certitude : {certainty}");
Console.WriteLine($"Pile ou face : {coinFlip}");

Certitude : [100 %:100]
Pile ou face : [50 %:200, 50 %:0]


## 3. Axiomes de Preferences Rationnelles

Von Neumann et Morgenstern (1944) ont etabli les axiomes qui definissent un **agent rationnel**.

### Notation

- A ≻ B : l'agent prefere strictement A a B
- A ≺ B : l'agent prefere strictement B a A  
- A ~ B : l'agent est indifferent entre A et B
- A ≿ B : l'agent prefere A ou est indifferent (preference faible)

### Les 4 Axiomes

#### Axiome 1 : Completude (Orderability)

> Pour toute paire de loteries A et B, exactement une relation est vraie :
> A ≻ B, ou B ≻ A, ou A ~ B

**Interpretation** : L'agent peut toujours comparer deux options.

#### Axiome 2 : Transitivite

> Si A ≻ B et B ≻ C, alors A ≻ C

**Interpretation** : Les preferences sont coherentes, pas de cycles.

#### Axiome 3 : Continuite

> Si A ≻ B ≻ C, alors il existe p ∈ (0,1) tel que :
> B ~ [p:A, (1-p):C]

**Interpretation** : Meme le pire outcome peut etre compense par une probabilite suffisante du meilleur.

#### Axiome 4 : Independance

> Si A ≻ B, alors pour toute loterie C et toute probabilite p :
> [p:A, (1-p):C] ≻ [p:B, (1-p):C]

**Interpretation** : Les preferences ne dependent pas des alternatives non pertinentes.

In [3]:
// Verification de la transitivite des preferences
// Simulons un agent avec une fonction d'utilite

Func<double, double> utilite = x => Math.Sqrt(x); // Utilite concave (aversion au risque)

double UtiliteEsperee(Loterie<double> L, Func<double, double> U)
{
    return L.Outcomes.Sum(o => o.prob * U(o.outcome));
}

// Trois loteries
var A = new Loterie<double>((1.0, 100));           // 100 certain
var B = new Loterie<double>((0.5, 196), (0.5, 0)); // E[B] = 98, E[U(B)] = 7
var C = new Loterie<double>((1.0, 49));            // 49 certain

double UA = UtiliteEsperee(A, utilite);
double UB = UtiliteEsperee(B, utilite);
double UC = UtiliteEsperee(C, utilite);

Console.WriteLine($"U(A) = {UA:F3} (100 certain)");
Console.WriteLine($"U(B) = {UB:F3} (50% de 196)");
Console.WriteLine($"U(C) = {UC:F3} (49 certain)");
Console.WriteLine();

Console.WriteLine($"A ≻ B ? {UA > UB} (transitivite: si A≻B et B≻C alors A≻C)");
Console.WriteLine($"B ≻ C ? {UB > UC}");
Console.WriteLine($"A ≻ C ? {UA > UC} (verifie !)");

U(A) = 10,000 (100 certain)
U(B) = 7,000 (50% de 196)
U(C) = 7,000 (49 certain)

A ≻ B ? True (transitivite: si A≻B et B≻C alors A≻C)
B ≻ C ? False
A ≻ C ? True (verifie !)


## 4. Theoreme de Representation

### Le theoreme fondamental

> **Theoreme (Von Neumann-Morgenstern, 1944)**
> 
> Si les preferences d'un agent satisfont les 4 axiomes, alors il existe une fonction U (unique a transformation affine pres) telle que :
> 
> A ≻ B  ⟺  E[U(A)] > E[U(B)]

### Consequences

1. **L'utilite esperee suffit** : Pour prendre des decisions rationnelles, il suffit de maximiser E[U(outcome)]

2. **Unicite relative** : U est unique a transformation affine pres (U' = aU + b, a > 0)

3. **Fondement de l'agent rationnel** : Un agent qui viole ces axiomes peut etre "money-pumped" (exploite)

### L'agent rationnel fonde sur l'utilite

```
Perception → Inference Bayesienne → Calcul E[U(action)] → Choix argmax → Action
```

L'agent :
1. Observe le monde (perception)
2. Met a jour ses croyances (inference bayesienne - ce que nous faisons avec Infer.NET)
3. Calcule l'utilite esperee de chaque action
4. Choisit l'action qui maximise l'utilite esperee

In [ ]:
// Modelisation d'un agent rationnel avec Infer.NET

// Scenario : Un patient a potentiellement une maladie.
// L'agent (medecin) doit decider : traiter ou ne pas traiter.

// Prior sur la maladie
Variable<bool> malade = Variable.Bernoulli(0.3).Named("malade");

// Test diagnostique (imparfait)
Variable<bool> testPositif = Variable.New<bool>().Named("test");
double sensibilite = 0.9;  // P(test+|malade)
double specificite = 0.8;  // P(test-|sain)

using (Variable.If(malade))
    testPositif.SetTo(Variable.Bernoulli(sensibilite));
using (Variable.IfNot(malade))
    testPositif.SetTo(Variable.Bernoulli(1 - specificite));

// Observation : test positif
testPositif.ObservedValue = true;

// Inference
InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

Bernoulli posteriorMalade = engine.Infer<Bernoulli>(malade);
Console.WriteLine($"Posterior P(malade|test+) = {posteriorMalade.GetProbTrue():P1}");

In [ ]:
// Maintenant, l'agent doit decider : traiter ou pas ?

// Matrice d'utilite (outcomes)
double U_traiter_malade = 80;    // Guerison (moins effets secondaires)
double U_traiter_sain = 60;      // Effets secondaires inutiles
double U_pas_traiter_malade = 0; // Catastrophe
double U_pas_traiter_sain = 100; // Parfait

double pMalade = posteriorMalade.GetProbTrue();
double pSain = 1 - pMalade;

// Utilite esperee de chaque action
double EU_traiter = pMalade * U_traiter_malade + pSain * U_traiter_sain;
double EU_pas_traiter = pMalade * U_pas_traiter_malade + pSain * U_pas_traiter_sain;

Console.WriteLine($"\nP(malade) = {pMalade:P1}, P(sain) = {pSain:P1}");
Console.WriteLine($"\nE[U(traiter)] = {pMalade:F2} x {U_traiter_malade} + {pSain:F2} x {U_traiter_sain} = {EU_traiter:F1}");
Console.WriteLine($"E[U(pas traiter)] = {pMalade:F2} x {U_pas_traiter_malade} + {pSain:F2} x {U_pas_traiter_sain} = {EU_pas_traiter:F1}");

string decision = EU_traiter > EU_pas_traiter ? "TRAITER" : "NE PAS TRAITER";
Console.WriteLine($"\nDecision optimale : {decision}");

## 5. Calibration par Mise a l'Indifference

### Le probleme pratique

Comment determiner la fonction d'utilite U d'un agent ?

### Methode de l'indifference (Axiome de continuite)

1. Fixez deux outcomes de reference : 
   - o_best (le meilleur) avec U(o_best) = 1
   - o_worst (le pire) avec U(o_worst) = 0

2. Pour chaque outcome o intermediaire, trouvez p* tel que :
   o ~ [p* : o_best, (1-p*) : o_worst]

3. Alors U(o) = p*

### Exemple : Evaluer une assurance

Vous avez une voiture de 10 000 EUR. Probabilite de vol = 5%.

| Outcome | Valeur |
|---------|--------|
| Pas de vol | 10 000 EUR |
| Vol | 0 EUR |

Une assurance coute 600 EUR. Devez-vous la prendre ?

In [5]:
// Calibration de l'utilite pour la decision d'assurance

// Supposons une utilite logarithmique (aversion au risque classique)
// U(x) = log(x + 1) normalise sur [0, 10000]

double Utilite(double x)
{
    if (x <= 0) return 0;
    return Math.Log(x + 1) / Math.Log(10001); // Normalise: U(10000) ≈ 1
}

// Sans assurance : loterie [0.95 : 10000, 0.05 : 0]
double pVol = 0.05;
double EU_sansAssurance = (1 - pVol) * Utilite(10000) + pVol * Utilite(0);

// Avec assurance : certitude de (10000 - 600) = 9400
double primeAssurance = 600;
double EU_avecAssurance = Utilite(10000 - primeAssurance);

Console.WriteLine($"E[U(sans assurance)] = 0.95 x U(10000) + 0.05 x U(0)");
Console.WriteLine($"                     = 0.95 x {Utilite(10000):F4} + 0.05 x {Utilite(0):F4}");
Console.WriteLine($"                     = {EU_sansAssurance:F4}");
Console.WriteLine();
Console.WriteLine($"E[U(avec assurance)] = U(9400) = {EU_avecAssurance:F4}");
Console.WriteLine();

if (EU_avecAssurance > EU_sansAssurance)
    Console.WriteLine($"Decision : PRENDRE l'assurance (gain utilite = {EU_avecAssurance - EU_sansAssurance:F4})");
else
    Console.WriteLine($"Decision : NE PAS prendre l'assurance");

// Trouver la prime maximale acceptable
// Chercher p tel que U(10000 - p) = EU_sansAssurance
// => 10000 - p = exp(EU_sansAssurance * log(10001)) - 1
double primeMax = 10000 - (Math.Exp(EU_sansAssurance * Math.Log(10001)) - 1);
Console.WriteLine($"\nPrime maximale acceptable : {primeMax:F0} EUR");

E[U(sans assurance)] = 0.95 x U(10000) + 0.05 x U(0)
                     = 0.95 x 1,0000 + 0.05 x 0,0000
                     = 0,9500

E[U(avec assurance)] = U(9400) = 0,9933

Decision : PRENDRE l'assurance (gain utilite = 0,0433)

Prime maximale acceptable : 3691 EUR


## 6. Modelisation avec Infer.NET

### Loteries comme modeles probabilistes

Une loterie peut etre modelisee comme une variable aleatoire dans Infer.NET.

In [ ]:
// Loterie : 50% de gagner 1000, 50% de perdre 500

Variable<bool> gagne = Variable.Bernoulli(0.5).Named("gagne");
Variable<double> gain = Variable.New<double>().Named("gain");

using (Variable.If(gagne))
{
    gain.SetTo(Variable.Constant(1000.0));
}
using (Variable.IfNot(gagne))
{
    gain.SetTo(Variable.Constant(-500.0));
}

// Inference du gain moyen (valeur esperee)
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

// Note: gain est un melange de deux points, pas une gaussienne
// Infer.NET retourne une approximation gaussienne
var gainDist = moteur.Infer<Gaussian>(gain);
Console.WriteLine($"Distribution du gain (approximation) : {gainDist}");
Console.WriteLine($"Valeur esperee : {gainDist.GetMean():F0} EUR");
Console.WriteLine();

// Calcul manuel de l'utilite esperee
// Avec U(x) = sqrt(x + 1000) (decalee pour eviter valeurs negatives)
Func<double, double> U = x => Math.Sqrt(x + 1000);

double EU = 0.5 * U(1000) + 0.5 * U(-500);
double U_certain = U(gainDist.GetMean()); // U(250)

Console.WriteLine($"E[U(loterie)] = 0.5 x U(1000) + 0.5 x U(-500)");
Console.WriteLine($"             = 0.5 x {U(1000):F2} + 0.5 x {U(-500):F2}");
Console.WriteLine($"             = {EU:F2}");
Console.WriteLine();
Console.WriteLine($"U(valeur esperee) = U(250) = {U_certain:F2}");
Console.WriteLine();
Console.WriteLine($"EU < U(EV) ? {EU < U_certain} (=> agent averse au risque prefere la certitude)");

## 7. Exercice : Calibrer Votre Fonction d'Utilite

### Enonce

Imaginez que vous devez choisir entre :

- **Option A** : Recevoir X EUR avec certitude
- **Option B** : 50% de chance de recevoir 1000 EUR, 50% de ne rien recevoir

**Question** : Pour quelle valeur de X etes-vous indifferent entre A et B ?

### Methode

1. Si votre X* = 500, vous etes **neutre au risque**
2. Si votre X* < 500, vous etes **averse au risque**
3. Si votre X* > 500, vous etes **amateur de risque**

### Code pour explorer

In [8]:
// Exploration : trouvez votre equivalent certain

// Supposons que votre equivalent certain est X* = 400 EUR
// Cela revele votre fonction d'utilite !

double equivalentCertain = 400; // <- Modifiez cette valeur selon votre preference

// Si vous etes indifferent entre X* certain et [0.5:1000, 0.5:0]
// Alors U(X*) = 0.5 * U(1000) + 0.5 * U(0)

// Normalisons : U(0) = 0, U(1000) = 1
// => U(X*) = 0.5

// Pour une utilite CRRA : U(x) = x^(1-rho) / (1-rho)
// Trouvons rho tel que U(400)/U(1000) = 0.5

double targetRatio = 0.5;

// Recherche binaire de rho
double rhoMin = 0.01, rhoMax = 5.0;
double rho = 1.0; // Initialisation

for (int i = 0; i < 50; i++)
{
    rho = (rhoMin + rhoMax) / 2;
    
    double Ux = Math.Pow(equivalentCertain, 1 - rho) / (1 - rho);
    double U1000 = Math.Pow(1000, 1 - rho) / (1 - rho);
    double ratio = Ux / U1000;
    
    if (ratio > targetRatio)
        rhoMin = rho;
    else
        rhoMax = rho;
}

Console.WriteLine($"Equivalent certain : {equivalentCertain} EUR");
Console.WriteLine($"Coefficient d'aversion au risque (rho) estime : {rho:F3}");
Console.WriteLine();

if (rho < 0.1)
    Console.WriteLine("Vous etes proche de la neutralite au risque.");
else if (rho < 1.0)
    Console.WriteLine("Vous avez une aversion au risque moderee.");
else if (rho < 2.0)
    Console.WriteLine("Vous avez une forte aversion au risque.");
else
    Console.WriteLine("Vous etes tres averse au risque.");

// Verification
Func<double, double> U_CRRA = x => Math.Pow(x, 1 - rho) / (1 - rho);
double EU_loterie = 0.5 * U_CRRA(1000) + 0.5 * U_CRRA(0.01); // Eviter log(0)
double U_cert = U_CRRA(equivalentCertain);

Console.WriteLine($"\nVerification : U({equivalentCertain}) = {U_cert:F4}");
Console.WriteLine($"             E[U(loterie)] ≈ {EU_loterie:F4}");

Equivalent certain : 400 EUR
Coefficient d'aversion au risque (rho) estime : 5,000

Vous etes tres averse au risque.

Verification : U(400) = -0,0000
             E[U(loterie)] ≈ -12500000,0000


## 8. Resume

| Concept | Description |
|---------|-------------|
| **Loterie** | Distribution de probabilite sur des outcomes |
| **Axiomes VNM** | Completude, Transitivite, Continuite, Independance |
| **Theoreme de representation** | Existence d'une fonction U telle que preferences = max E[U] |
| **Agent rationnel** | Maximise l'utilite esperee de ses actions |
| **Calibration** | Methode de l'indifference pour determiner U |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Approfondir l'aversion au risque | [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb) |
| Decisions multi-criteres | [Infer-16-Decision-Multi-Attribute](Infer-16-Decision-Multi-Attribute.ipynb) |
| Visualiser les reseaux de decision | [Infer-17-Decision-Networks](Infer-17-Decision-Networks.ipynb) |

---

## Prochaine etape

Dans [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb), nous approfondirons :

- Le paradoxe de Saint-Petersbourg
- Les fonctions d'utilite CARA et CRRA
- Les coefficients d'aversion au risque (Arrow-Pratt)
- La dominance stochastique

---

## References

- Von Neumann & Morgenstern (1944) : Theory of Games and Economic Behavior
- Russell & Norvig : Artificial Intelligence, Chapter 16
- Bernoulli (1738) : Specimen Theoriae Novae de Mensura Sortis (Paradoxe de St-Petersbourg)